other llama code :https://colab.research.google.com/drive/1wvpt2HnWu-AsGp3zCJosqxyvI3DiRz0q#scrollTo=JiDxJlPJKSbS

In [ ]:
!pip install openai
!pip install langchain
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.8/220.8 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.2/174.2 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.2 MB/s

In [ ]:
!pip install datasets

In [ ]:
import pandas as pd
import numpy as np
import csv
from openai import OpenAI
from openai._client import OpenAI

In [ ]:
client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key="OPENAI_API_KEY",
)

In [ ]:
import os
import openai
os.environ['OPENAI_API_KEY'] = "OPENAI_API_KEY"
if 'OPENAI_API_KEY' in os.environ:
    print("API Key set successfully.")
else:
    print("Failed to set API Key.")

API Key set successfully.


# Loading hatespeech dataset from GAB

In [ ]:
# file_loc = "/home/local/ASURITE/anirmal1/Downloads/gab_dataset.csv"
file_loc = 'youtube_dataset.csv'
dataset = pd.read_csv(file_loc)
# dataset
dataset

,text,label,exp_split
0,I wonder if isis will respond?,0,train
1,WAIT WHY ISN'T MULAN IRANIAN? WHERE'S THE REPR...,0,train
2,Can you please change the title of this video ...,0,train
3,i hate illegal immigrants. i love legal immigr...,1,train
4,+andy artin I believe the Lord wants you to ca...,0,train
...,...,...,...
5047,u cannot do anything but u can report each you...,0,test
5048,Zionist Jew Tube has made changes to it's algo...,0,test
5049,FUK THESE 2 FACED BRITISH BASTARDS...FUK THE F...,1,test
5050,"You don't have ""peace"" in Europe or Germany. Y...",0,test


## Prompting the API

In [ ]:
def get_rationales(prompt, model="gpt-3.5-turbo-0613"):
    response = client.chat.completions.create(
    messages=[
    {
        "role": "system",
        "content": "Follow the instructions as closely as possible. Output exactly in the format that is specified by the user."
    },
    {
        "role": "user",
        "content": "You are a content moderation bot. Identify the list of rationales, list of derogatory language, list of cuss words that promote a hateful sentiment and respond with non-hateful if there are none. Note: The output should be in a json format.\nYou should then generate a JSON output structured like:\n{\n  \"text\": \"[original text]\",\n  \"rationales\": [\n    \"[identified targets/accusations]\",\n    \"[identified targets/accusations]\" \n  ],\n  \"derogatory_language\": [\n    \"[identified terms]\",\n    \"[identified terms]\"\n  ],\n  \"cuss_words\": [\n    \"[cuss word 1]\",\n    \"[cuss word 2]\"\n  ]\n}\n"
    },
    {
        "role": "assistant",
        "content": "Text: always thought it astounding no policing agency dares do this to the <number> armed muslim compounds all over the country"
    },
    {
        "role": "assistant",
        "content": "{\n  \"text\": \"always thought it astounding no policing agency dares do this to the <number> armed muslim compounds all over the country\",\n  \"rationales\": [\n    \"armed muslim compounds\"\n  ],\n  \"derogatory_language\": [\n    \"muslim\"\n  ],\n  \"cuss_words\": []\n}"
    },
    {
        "role": "user",
        "content": prompt
    }
    ],
    model=model,
    temperature=0.1,
    max_tokens=500,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
    )
    # return response.choices[0].message["content"]
    return response.choices[0].message.content

# Testing for one example

In [ ]:
prompt = f'''{
dataset['text'][5]
}'''

In [ ]:
prompt

'I hope you all will become Muslim'

In [ ]:
response = get_rationales(prompt)

In [ ]:
prompt

'I hope you all will become Muslim'

In [ ]:
response

'{\n  "text": "I hope you all will become Muslim",\n  "rationales": [],\n  "derogatory_language": [],\n  "cuss_words": []\n}'

# Testing for all samples

In [ ]:
import csv

# Creating a new CSV file to write the updated data
updated_file_path = 'CHATGPT_YOUTUBE_dataset.csv'

with open(updated_file_path, mode='w', newline='') as updated_file:
    csv_writer = csv.writer(updated_file)

    # Writing the header row
    csv_writer.writerow(dataset.columns.tolist() + ['ChatGPT Response'])

    for index, row in dataset.iterrows():
        tweet = row['text']
        try:
            response = get_rationales(tweet)
        except:
            response = "Language model fails to give output for this text."

        row_values = row.tolist()
        row_values.append(response)
        csv_writer.writerow(row_values)

# Reading the updated data into a DataFrame
updated_data = pd.read_csv(updated_file_path)


In [ ]:
updated_data

,text,label,exp_split,ChatGPT Response
0,I wonder if isis will respond?,0,train,"{\n ""text"": ""I wonder if isis will respond?"",..."
1,WAIT WHY ISN'T MULAN IRANIAN? WHERE'S THE REPR...,0,train,"{\n ""text"": ""WAIT WHY ISN'T MULAN IRANIAN? WH..."
2,Can you please change the title of this video ...,0,train,"{\n ""text"": ""Can you please change the title ..."
3,i hate illegal immigrants. i love legal immigr...,1,train,"{\n ""text"": ""i hate illegal immigrants. i lov..."
4,+andy artin I believe the Lord wants you to ca...,0,train,"{\n ""text"": ""+andy artin I believe the Lord w..."
...,...,...,...,...
5047,u cannot do anything but u can report each you...,0,test,"{\n ""text"": ""u cannot do anything but u can r..."
5048,Zionist Jew Tube has made changes to it's algo...,0,test,"{\n ""text"": ""Zionist Jew Tube has made change..."
5049,FUK THESE 2 FACED BRITISH BASTARDS...FUK THE F...,1,test,"{\n ""text"": ""FUK THESE 2 FACED BRITISH BASTAR..."
5050,"You don't have ""peace"" in Europe or Germany. Y...",0,test,"{\n ""text"": ""You don't have \""peace\"" in Euro..."


In [ ]:
# updated_data.to_csv('FINAL_CHATGPT_YOUTUBE_dataset_answers_all_dataset.csv')

In [ ]:
from google.colab import files
files.download('CHATGPT_YOUTUBE_dataset.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>